In [1]:
from sys import path

import numpy as np
import pandas as pd 
from scipy.stats import norm
import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set_theme()

%load_ext autoreload
%autoreload 2

# user-written 
import w8_estimation as est 
import w8_LinearModel as lm
import w8_probit as probit
import w8_logit as logit
import marginal_effects as me


In [2]:
# Outcome label
y_lab = 'anyuseofforce_coded'

# Dataset columns
rawdat_columns = [
    'anyuseofforce_coded',
    
    # Subject (civilian) characteristics
    'sblack',
    'shisp',
    'swhite',
    'smale',
    'sother', 
    'sage',
    'sempl', 
    'sincome',
    'spop', 
    'sbehavior',
    
    # Officer characteristics
    'omajblack',
    'omajhisp',
    'omajwhite',
    'omajother', 
    'osplit', 
    
    # Encounter characteristics
    'daytime',
    'inctype_lin', 
    'year'
]


In [3]:
# Final X-matrix variable labels 
x_lab = [
    # Subject vars (white is reference)
    'sblack',
    'shisp',
    'smale',
    'sage',

    # Officer vars (white is reference)
    #'omajblack', --> Udelukkes fordi der ikke er nogen
    'omajhisp',
    #'omajother',

    # Encounter vars
    'daytime'
]

In [4]:
dat = pd.read_csv('ppcs_cc.csv')

N = dat.shape[0]

# reorder columns 
dat = dat[[y_lab] + x_lab].copy()

dat.head(5)

assert dat.notnull().all(axis=1).all(), 'Missings in the dataset, take them out!'

In [5]:
y = dat[y_lab].values
x = dat[x_lab].values
K = x.shape[1]

print("Shape x:", x.shape)
print("Rank x:", np.linalg.matrix_rank(x))
y.shape

Shape x: (3799, 6)
Rank x: 6


(3799,)

In [6]:
# OLS estimates
ols_results = lm.estimate(y, x, robust_se=True)
ols_tab = lm.print_table((y_lab, x_lab), ols_results, title='LPM results')
ols_tab

LPM results
Dependent variable: anyuseofforce_coded

R2 = 0.004
sigma2 = nan


,b_hat,se,t
sblack,0.0048,0.0042,1.1465
shisp,0.0126,0.0063,1.9881
smale,0.0068,0.0022,3.0771
sage,-0.0000,0.0000,-0.4581
omajhisp,0.0042,0.0107,0.3885
daytime,-0.0010,0.0022,-0.4525


In [7]:
theta0 = probit.starting_values(y, x)
theta0.ndim==1

True

In [8]:
ll = probit.loglikelihood(theta0, y, x)
np.isclose(np.mean(ll), -1.0411283428047824)

False

In [9]:
#probit_results = est.estimate(probit.q, theta0, y, x)

In [10]:
#probit_tab = est.print_table(x_lab, probit_results, title=f'Probit, y = {y_lab}')
#probit_tab

In [11]:
theta0 = logit.starting_values(y, x)
theta0 

array([ 1.90417771e-02,  5.02863186e-02,  2.73331524e-02, -7.37894881e-05,
        1.66389576e-02, -4.05348732e-03])

In [12]:
ll = logit.loglikelihood(theta0, y, x)
np.isclose(np.mean(ll),-0.9974267061091704)

False

In [13]:
logit_results = est.estimate(logit.q, theta0, y, x)

Optimization terminated successfully.
         Current function value: 0.033326
         Iterations: 65
         Function evaluations: 574
         Gradient evaluations: 82


In [14]:
logit_tab = est.print_table(x_lab, logit_results, title=f'Logit, y = {y_lab}')
logit_tab

Optimizer succeeded after 65 iter. (574 func. evals.). Final criterion:  0.03333.
Logit, y = anyuseofforce_coded


,theta,se,t
sblack,0.1378,0.7223,0.1907
shisp,0.6087,0.5319,1.1443
smale,0.0021,0.6597,0.0032
sage,-0.1628,0.0168,-9.6694
omajhisp,0.3236,1.3030,0.2484
daytime,-0.9576,0.5766,-1.6607


In [15]:
# Let us make a vector of the values we want to investigate
x_me = np.array([0, 0, 1, 25, 1, 1], dtype=float).reshape(1, K)  # recall, x_i is a row vector
pd.DataFrame(x_me, columns=x_lab, index=['x_me'])  # print it out


,sblack,shisp,smale,sage,omajhisp,daytime
x_me,0.0,0.0,1.0,25.0,1.0,1.0


In [16]:
# Delta-method partial effects for logit
b_lg = logit_results['theta']
cov_lg = logit_results['cov']

idx_sblack = x_lab.index('sblack')
idx_shisp = x_lab.index('shisp')
idx_sage = x_lab.index('sage')

x_sblack = x_me.copy()
x_sblack[:, idx_sblack] = 1
x_shisp = x_me.copy()
x_shisp[:, idx_shisp] = 1

me_sblack, se_sblack = me.discrete_effect_delta(b_lg, cov_lg, x_me, x_sblack, logit.G)
me_shisp, se_shisp = me.discrete_effect_delta(b_lg, cov_lg, x_me, x_shisp, logit.G)
me_sage, se_sage = me.continuous_effect_delta(b_lg, cov_lg, x_me, idx_sage, logit.Gprime)

ols_effects = ols_results['b_hat'][[idx_sblack, idx_shisp, idx_sage]]

me_tab = pd.DataFrame({
    'Var': ['sblack', 'shisp', 'sage'],
    'Effect (LPM)': ols_effects,
    'Effect (Logit)': [me_sblack, me_shisp, me_sage],
    's.e. (Logit)': [se_sblack, se_shisp, se_sage],
}).set_index('Var')
me_tab['t (Logit)'] = me_tab['Effect (Logit)'] / me_tab['s.e. (Logit)']


In [17]:
me_tab.round(4)


,Effect (LPM),Effect (Logit),s.e. (Logit),t (Logit)
Var,,,,
sblack,0.0048,0.0013,0.0074,0.1778
shisp,0.0126,0.0074,0.0091,0.8156
sage,-0.0000,-0.0015,0.0020,-0.7393
